<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>



<a target="_blank" href="https://colab.research.google.com/github/CienciaDeDatosEspacial/GeoDataFrame_SpatialOperation/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Basic spatial operations in  Geo Dataframes

We will review some important operations for geodataframes.

Let's remember the contents of the world map from last session:

<a class="anchor" id="0"></a>

In [ ]:
linkWorldMap="https://github.com/CienciaDeDatosEspacial/dataSets/raw/refs/heads/main/WORLD/worldMaps.gpkg"

import geopandas as gpd
gpd.list_layers(linkWorldMap)

Let's open all the layers (this takes a minute):

In [ ]:
countries=gpd.read_file(linkWorldMap,layer='countries')
rivers=gpd.read_file(linkWorldMap,layer='rivers')
cities=gpd.read_file(linkWorldMap,layer='cities')



Now, let's see some important spatial operations.


<a class="anchor" id="1"></a>

# Subsetting

## Filtering

You can keep some elements by subsetting by *filtering*, as we used to do in common pandas data frames.

In [ ]:
countries.head()

In [ ]:
# as DF
countries.iloc[50:,]

In [ ]:
# as DF
countries.loc[50:,'geometry']

But as a GeDF, you can also filter using a coordinate point via __cx__. Let me get the bounding box of the map:

In [ ]:
countries.total_bounds

As you are getting __[minx, miny, maxx, maxy]__ let me select a valid coordinate, i.e. (0,0)

In [ ]:
countries.cx[:0,:0]

In [ ]:
#then
countries.cx[:0,:0].plot()

Notice __cx__ would be cleaner if spatial element is a point.

## Clipping

Let me keep one country:

In [ ]:
brazil=countries[countries.COUNTRY=='Brazil']
#see
brazil

Pay attention to this GDF:

In [ ]:
cities

The GDF has a column 'COUNTRY' too.

Now, check the rivers GDF:

In [ ]:
rivers

As you see, this GDF has no Country. But since it has geometry, you can keep the rivers, or their sections, that serve a country:

In [ ]:
riversBrazil_clipped = gpd.clip(gdf=rivers,
                               mask=brazil)

Then, you can plot the clipped version:

In [ ]:
base = brazil.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
riversBrazil_clipped.plot(edgecolor='blue', linewidth=0.5,
                    ax=base)

The geometry types are not modified:

In [ ]:
set(brazil.geom_type), set(riversBrazil_clipped.geom_type)



_____________


<a class="anchor" id="3"></a>

# UNARY Operations on GeoDF

Let me bring the projected data from Brazil.

In [ ]:
LinkBrazil="https://github.com/CienciaDeDatosEspacial/dataSets/raw/refs/heads/main/BRAZIL/brazil_5880.gpkg"
## we have
gpd.list_layers(LinkBrazil)

Let me open municipalities:

In [ ]:
brazil_municipalities=gpd.read_file(LinkBrazil,layer='municipalities')
brazil_municipalities.plot(facecolor='lightgrey', edgecolor='black',linewidth=0.2)

In [ ]:
#see
brazil_municipalities.head()

In [ ]:
## we have
len(brazil_municipalities.ADM2_PT)

In [ ]:
# higher level count
len(set(brazil_municipalities.ADM1_PT))

Then, this is Minas Gerais:

In [ ]:
brazil_municipalities[brazil_municipalities.ADM1_PT=='Minas Gerais'].plot(edgecolor='yellow')

## I. Operation that combine 

Let's see the options to combine:

### Unary UNION

We can combine all these polygons into one:

In [ ]:
brazil_municipalities[brazil_municipalities.ADM1_PT=='Minas Gerais'].union_all()

Let's save that result:

In [ ]:
MinasGerais_union=brazil_municipalities[brazil_municipalities.ADM1_PT=='Minas Gerais'].union_all()

In [ ]:
# what do we have?
type(MinasGerais_union)

You can turn that shapely object into a GeoDF like this:

In [ ]:
gpd.GeoDataFrame(index=[0], # one element
                 data={'ADM':'Minas Gerais'}, # the column and the value
                 crs=brazil_municipalities.crs,
                 geometry=[MinasGerais_union]) # the recent union

<a class="anchor" id="21"></a>

### Dissolve

#### a. Dissolve as Union
Using  **dissolve** is an alternative to _UNION_:

In [ ]:
brazil_municipalities[brazil_municipalities.ADM1_PT=='Minas Gerais'].dissolve().plot()

Let me save the result, and see the type :

In [ ]:
MinasGerais_dissolve=brazil_municipalities[brazil_municipalities.ADM1_PT=='Minas Gerais'].dissolve()

# we got?
type(MinasGerais_dissolve)

You got a GEOdf this time:

In [ ]:
## see
MinasGerais_dissolve

In [ ]:
# keeping what is relevant
MinasGerais_dissolve.drop(columns=['ADM2_PT','ADM2_PCODE','ET_ID'],inplace=True)

# then
MinasGerais_dissolve

#### b. Dissolve for groups

Using _dissolve()_ with no arguments returns the union of the polygons, BUT also you get a GEOdf.
However, if you have a column that represents a grouping (as we do), you can dissolve by that column:

In [ ]:
# dissolving
brazil_municipalities.dissolve(by='ADM1_PT').plot(facecolor='lightgrey', edgecolor='black',linewidth=0.2)

Again, let me save this result:

In [ ]:
Brazil_adm1_diss=brazil_municipalities.dissolve(by='ADM1_PT')

We know we have a GeoDF; let's see contents:

In [ ]:
Brazil_adm1_diss.head()

Again, we can drop columns that do not bring important information:

In [ ]:
Brazil_adm1_diss.drop(columns=['ADM2_PT','ADM2_PCODE','ET_ID'],inplace=True)
Brazil_adm1_diss.reset_index(inplace=True)
Brazil_adm1_diss.info()

#### c. Dissolve and aggregate

In pandas, you can aggregate data using some statistics. Let me open the map with indicators we created in a previous session:

In [ ]:
linkInd="https://github.com/CienciaDeDatosEspacial/dataSets/raw/refs/heads/main/WORLD/worldindicators.json"
indicators=gpd.read_file(linkInd)
indicators.head()

You can compute the mean of the countries by region, using a DF approach like this:

In [ ]:
indicators.groupby('region').agg({'fragility':'mean'})


The RESULT is just data, you see no spatial information. It got lost.

The appropriate operation to conserve spatial information is **Dissolve**:

In [ ]:
indicatorsByRegion=indicators.dissolve(
     by="region",
     aggfunc={
         "fragility": ["mean"]},as_index=False, grid_size=0.1
 )

## see the spatial info
indicatorsByRegion

Without renaming, you can request a choropleth:

In [ ]:
# !pip install mapclassify

In [ ]:
indicatorsByRegion.plot(column =('fragility', 'mean'),scheme='quantiles', cmap='OrRd_r',
                        legend=True,
                        legend_kwds={"title": "Fragility",'loc': 'lower left'},
                        edgecolor='black',linewidth=0.2,
                        figsize=(15, 10))

Keep in mind that the combining of objects via UNION_ALL and DISSOLVE are destructive, we can not undo them. We have operations like EXPLODE that work in the reverse direction (splitting) but even those can not undo the output of UNION_ALL and DISSOLVE. Always preserve your original GeoDataFrame before using these operations, as they permanently alter your data in ways that cannot be reversed.

In [ ]:
# Italy, for example is a multipolygon
countries[countries.COUNTRY=='Italy']

In [ ]:
countries[countries.COUNTRY=='Italy'].plot()

In [ ]:
# here the mainland is separated from Sardinia, and Sicily
countries[countries.COUNTRY=='Italy'].explode()

_____________


<a class="anchor" id="4"></a>

## II. The convex hull

Some time you may have the need to create a polygon that serves as an envelope to a set of points.

For this example, let me keep the large airports:

In [ ]:
# just the union
airports_5880=gpd.read_file(LinkBrazil,layer='airports')
large_airport=airports_5880[airports_5880.kind=='large_airport']
large_airport.plot()

May I use now **convex_hull**?

In [ ]:
## you see no difference!!
large_airport.convex_hull.plot()

The objects to be enveloped required to be **combined** previously: 

In [ ]:
# hull of the union
large_airport.union_all().convex_hull

We got:

In [ ]:
# this geometry not a GeoDF...yet
type(large_airport.union_all().convex_hull)

Let's turn this geometry into a GDF:

In [ ]:
LargeAirport_hull= gpd.GeoDataFrame(index=[0],
                                    crs=large_airport.crs,
                                    geometry=[large_airport.union_all().convex_hull])
LargeAirport_hull['name']='large airports hull' # optional

# then

LargeAirport_hull

Let's use the GDF in plotting:

In [ ]:
brazil_5880=gpd.read_file(LinkBrazil,layer='country')
base=brazil_5880.plot(facecolor='yellow')
large_airport.plot(ax=base)
LargeAirport_hull.plot(ax=base,facecolor='green',
                       edgecolor='white',alpha=0.4,
                       hatch='X')

You can get a convex hull of lines or polygons:

In [ ]:
riversBrazil_clipped.union_all().convex_hull

You can use it for dissolved polygons:

In [ ]:
MinasGerais_dissolve.plot()

In [ ]:
#then
MinasGerais_dissolve.convex_hull.plot()

Remember that **union_all** and **dissolve()** give different outputs:

In [ ]:
# you got a series, not just a geometry 
type(MinasGerais_dissolve.convex_hull)

In [ ]:
# a simple "to_frame" does the job
MinasGerais_dissolve.convex_hull.to_frame()

In [ ]:
# more details
MinasGerais_hull=MinasGerais_dissolve.convex_hull.to_frame()
MinasGerais_hull["name"]="Minas Gerais"
MinasGerais_hull.rename(columns={0:"geometry"})
MinasGerais_hull
                        

In [ ]:
#noticed the crs was inherired
MinasGerais_hull.crs

You need the union/dissolve to avoid that a hull were created  for each row (polygon here), see:

In [ ]:
#original not COMBINED:
Brazil_adm1_diss.plot(edgecolor="yellow")

In [ ]:
# hull of Non combined
Brazil_adm1_diss.convex_hull.plot(edgecolor="yellow")

In [ ]:
# the hull of Brazil
Brazil_adm1_diss.dissolve().convex_hull.plot(edgecolor="yellow")

## III. The Buffer

The buffer will create a polygon that follows the same shape of the original vector (line, polygon, point).

Here, from the last polygon dissolved, we create a buffer (50,000 mts):

In [ ]:
Brazil_adm1_diss.dissolve().buffer(50000).plot(facecolor="yellow")

In [ ]:
#you will see the buffer clearer now

base=Brazil_adm1_diss.dissolve().buffer(50000).plot(facecolor="yellow")
Brazil_adm1_diss.dissolve().plot(ax=base)

Let me buffer the Brazil rivers:

In [ ]:
# this is the original
riversBrazil_clipped.plot()

But, verify crs as we are going to use distances:

In [ ]:
riversBrazil_clipped.crs

Let's reproject:

In [ ]:
riversBrazil_5880=riversBrazil_clipped.copy()
riversBrazil_5880 = riversBrazil_5880.to_crs('EPSG:5880')

Now I can use the rivers to create a buffer of 50000 meters:

In [ ]:
# 50000 at each side (radius)
riversBrazil_5880.buffer(50000).plot(facecolor='yellow', edgecolor='black',linewidth=0.2)

The resulting buffer is:

In [ ]:
type(riversBrazil_5880.buffer(50000))

Then:

In [ ]:
base=riversBrazil_5880.buffer(50000).plot(facecolor='yellow',edgecolor='black',linewidth=0.2)
riversBrazil_5880.plot(ax=base)

notice:

In [ ]:
riv_buf_right = riversBrazil_5880.buffer(distance = 50000, single_sided = True)
riv_buf_left = riversBrazil_5880.buffer(distance = -25000, single_sided = True)

base =riv_buf_right.plot(color='green')
riv_buf_left.plot(ax=base, color='purple')

Let me save the rivers reprojected in a JSON file:

In [ ]:
riversBrazil_5880.to_file("riversBrazil_5880.geojson", driver="GeoJSON")


_____________

<a class="anchor" id="5"></a>
# BINARY Operations: Spatial Overlay

We might need to create or find some geometries from the geometries we already have. Using a set theory approach, we will see the use of _intersection_, _union_, _difference_, and _symmetric difference_.

Let me first divide Brazil using its centroid:

In [ ]:
# coordinates
centroidX,centroidY=brazil_5880.centroid.x.values[0],brazil_5880.centroid.y.values[0]

In [ ]:
brazil_states=gpd.read_file(LinkBrazil,layer='states')

In [ ]:
# the north
N_brazil=brazil_states.cx[:,centroidY:]
# the south
S_brazil=brazil_states.cx[:,:centroidY]
# the west
W_brazil=brazil_states.cx[:centroidX,:]
# the east
E_brazil=brazil_states.cx[centroidX:,:]

In [ ]:
N_brazil

In [ ]:
S_brazil

In [ ]:
base= N_brazil.plot(facecolor='yellow', edgecolor='black',linewidth=0.2, alpha=0.6)
S_brazil.plot(facecolor='grey', edgecolor='black',linewidth=0.2,ax=base, alpha=0.4)

In [ ]:
set(S_brazil.ADM1_PT) & set(N_brazil.ADM1_PT)

In [ ]:
set(E_brazil.ADM1_PT) & set(W_brazil.ADM1_PT)

In [ ]:
base=E_brazil.plot(facecolor='yellow', edgecolor='black',linewidth=0.2, alpha=0.6)
W_brazil.plot(facecolor='grey', edgecolor='black',linewidth=0.2,ax=base, alpha=0.4)

## Intersection

We keep what is common between GeoDFs:

In [ ]:
NS_brazil=N_brazil.overlay(S_brazil, how="intersection",keep_geom_type=True)
# see results
NS_brazil

Notice the intersection was not clean, you have three more polygons:

In [ ]:
NS_brazil[NS_brazil.ADM1_PT_1!= NS_brazil.ADM1_PT_2]

This is the amount of area that is in fact a topological problem:

In [ ]:
NS_brazil[NS_brazil.ADM1_PT_1!= NS_brazil.ADM1_PT_2].geometry.area.sum()

This represents the area with topologically valid boundaries:

In [ ]:
NS_brazil[NS_brazil.ADM1_PT_1== NS_brazil.ADM1_PT_2].geometry.area.sum()

A way to measure the share of the low quality:

In [ ]:
NS_brazil[NS_brazil.ADM1_PT_1!= NS_brazil.ADM1_PT_2].geometry.area.sum()/  \
NS_brazil[NS_brazil.ADM1_PT_1== NS_brazil.ADM1_PT_2].geometry.area.sum() #continues from above

So, spatial overlay operations do their best to give you true results; but unfortunately, as the quality of the sources is not perfect, you may get messy results. It is our job to detect and make decisions. Let's keep two GeoDF, one with the unperfect result, and another with the true output.

In [ ]:
NS_brazil_messy=NS_brazil.copy()
NS_brazil=NS_brazil[NS_brazil.ADM1_PT_1== NS_brazil.ADM1_PT_2]

This should be what we expected to see:

In [ ]:
NS_brazil

The clean data has minor things to improve, delete redundant columns, and reset the index so they are a correlative sequence. 

In [ ]:
# avoid redundancy
keep=['ADM0_EN_1','ADM1_PT_1','geometry']
NS_brazil=NS_brazil.loc[:,keep]

# reset for correlative sequence
NS_brazil.reset_index(drop=True, inplace=True)

Based on the previous case, we may expect a similar situation here:

In [ ]:
# keeping the overlay
WE_brazil=W_brazil.overlay(E_brazil, how="intersection",keep_geom_type=True)
WE_brazil[WE_brazil.ADM1_PT_1!= WE_brazil.ADM1_PT_2]

Let's do the same as before:

In [ ]:
WE_brazil_messy=WE_brazil.copy()
WE_brazil=WE_brazil[WE_brazil.ADM1_PT_1== WE_brazil.ADM1_PT_2]

keep=['ADM0_EN_1','ADM1_PT_1','geometry']
WE_brazil=WE_brazil.loc[:,keep]

WE_brazil.reset_index(drop=True, inplace=True)

## Union

Different from UNION_ALL (which acts as DISSOLVE), here we will combine two GeoDFs. 

In [ ]:
NS_brazil.info()

In [ ]:
WE_brazil.info()

In [ ]:
# now
NS_brazil.overlay(WE_brazil,how="union",keep_geom_type=True)

As you see, geometries are fine, but not attributes. It is strictly NOT appending the GeoDFs:

In [ ]:
# appending
import pandas as pd

pd.concat([NS_brazil,WE_brazil],ignore_index=True).drop_duplicates(subset='geometry')

Let me create an object to save the previous result:

In [ ]:
MidBrazil=pd.concat([NS_brazil,WE_brazil],ignore_index=True).drop_duplicates(subset='geometry').dissolve()
MidBrazil

In [ ]:
# some cleaning

MidBrazil['region']='center'
MidBrazil.rename(columns={'ADM0_EN_1':'country'},inplace=True)
MidBrazil=MidBrazil.loc[:,['country','region','geometry']]

MidBrazil

In [ ]:
# see it
base=brazil_5880.plot(facecolor='yellow')
MidBrazil.plot(ax=base)

## Difference

Here, you keep what belongs to the GeoDF to left that is not in the GeoDF to the right:

In [ ]:
# we keep nothern states that are not in the 'S_brazil' region
N_brazil.overlay(S_brazil, how='difference')

We got a clean result. Let's plot it:

In [ ]:
base=N_brazil.plot(color='yellow', edgecolor='black',alpha=0.1)
N_brazil.overlay(S_brazil, how='difference').plot(ax=base)

Keep in mind that **difference** is not commutative:

In [ ]:
S_brazil.overlay(N_brazil, how='difference')

In [ ]:
base=N_brazil.plot(color='yellow', edgecolor='black',alpha=0.1)
S_brazil.overlay(N_brazil, how='difference').plot(ax=base)

## Symmetric Difference

This is the opposite to *intersection*, you keep what is not in the intersection. Notice that this operation is commutative!

In [ ]:
N_brazil.overlay(S_brazil, how='symmetric_difference')

This operation gave a clean result again. Let's plot it:

In [ ]:
N_brazil.overlay(S_brazil, how='symmetric_difference').plot()

What if we split brazil in halves?

In [ ]:
brazil_total=brazil_states.dissolve()
bbox = brazil_total.total_bounds # the bounding box
centroid_y = brazil_total.centroid.y.iloc[0] # the y of centroid

minx, miny, maxx, maxy = bbox # the bounding box coordinates

# new boxes
north_bbox_tuple = (minx, centroid_y, maxx, maxy)
south_bbox_tuple = (minx, miny, maxx, centroid_y)

# split Brazil
north_gdf = brazil_total.clip(north_bbox_tuple)
south_gdf = brazil_total.clip(south_bbox_tuple)

# Add region identifiers
north_gdf['region'] = 'north'
south_gdf['region'] = 'south'

In [ ]:
base=north_gdf.plot(color='grey')
south_gdf.plot(ax=base)

Now, you can apply overlays and get portions of states:

In [ ]:
N_brazil.overlay(north_gdf, how='symmetric_difference',keep_geom_type=False).plot(color='grey')

Notice **keep_geom_type=False**. This operation may produce geometries other than polygons:


In [ ]:
N_brazil.overlay(north_gdf, how='symmetric_difference',keep_geom_type=False).geom_type

Take a look:

In [ ]:
N_brazil.overlay(north_gdf, how='symmetric_difference',keep_geom_type=False).explore()


_____________

<a class="anchor" id="6"></a>

# Validity of Geometry

Geometries are created in a way that some issues may appear, especially in (multi) polygons.
Let's check if our recent maps on states and municipalities are valid:

In [ ]:
# non valid
MunisS_brazil[~MunisS_brazil.is_valid]

In [ ]:
# see the invalid:
MunisS_brazil[~MunisS_brazil.is_valid].plot()

It is difficult to see what is wrong. Let's get some information:

In [ ]:
# what is wrong?

from shapely.validation import explain_validity, make_valid

explain_validity(MunisS_brazil[~MunisS_brazil.is_valid].geometry)

In [ ]:
# varieties?
MunisS_brazil['validity']=[x.split('[')[0] for x in MunisS_brazil.geometry.apply(lambda x: explain_validity(x))]
MunisS_brazil['validity'].value_counts()

In [ ]:
# solving the issue:
MunisS_brazil.drop(columns=['validity'],inplace=True)

MunisS_brazil_valid=MunisS_brazil.copy()

MunisS_brazil_valid['geometry'] = [make_valid(row)  if not row.is_valid else row for row in MunisS_brazil_valid['geometry'] ]
#any invalid?
MunisS_brazil_valid[~MunisS_brazil_valid.is_valid]

The _solution_ we got may help for some advanced techniques, but may also give us some extra trouble. Notice that once geopandas solved the problem, you  have created **collections**:

In [ ]:
pd.Series([type(x) for x in MunisS_brazil_valid.geometry]).value_counts()

## Buffers and Validity

The buffering process helps cleaning simple invalidities:

In [ ]:
MunisS_brazil_valid=MunisS_brazil.copy()

MunisS_brazil_valid['geometry'] = MunisS_brazil_valid['geometry'].buffer(0)

#any invalid?
MunisS_brazil_valid[~MunisS_brazil_valid.is_valid]

Comparing with the previous result, this time you got no collections:

In [ ]:
# then:
pd.Series([type(x) for x in MunisS_brazil_valid.geometry]).value_counts()

This 'buffer trick' may not always work:

In [ ]:
# previously
indicators.dissolve(
     by="region",
     aggfunc={
         "fragility": ["mean"]},as_index=False
 ).plot(column =('fragility', 'mean'),scheme='quantiles', cmap='OrRd_r',
                        legend=True,
                        legend_kwds={"title": "Fragility",'loc': 'lower left'},
                        edgecolor='black',linewidth=0.2,
                        figsize=(15, 10))

Notice that AFRICA has some lines that should have dissappeared after dissolving, but you can still see some lines.

We could try the buffer trick:

In [ ]:
indicators_valid=indicators.copy()
indicators_valid['geometry'] = indicators_valid['geometry'].buffer(0)
dissolved_gdf=indicators_valid.dissolve(
     by="region",
     aggfunc={
         "fragility": ["mean"]},as_index=False, grid_size=0.1
 )




dissolved_gdf.plot(column =('fragility', 'mean'),scheme='quantiles', cmap='OrRd_r',
                        legend=True,
                        legend_kwds={"title": "Fragility",'loc': 'lower left'},
                        edgecolor='black',linewidth=0.2,
                        figsize=(15, 10))

It did not work either. We may use a new functionality in GeoPandas's dissolve:

In [ ]:
dissolved_gdf=indicators.dissolve(
     by="region",
     aggfunc={
         "fragility": ["mean"]},as_index=False, grid_size=0.1
 )




dissolved_gdf.plot(column =('fragility', 'mean'),scheme='quantiles', cmap='OrRd_r',
                        legend=True,
                        legend_kwds={"title": "Fragility",'loc': 'lower left'},
                        edgecolor='black',linewidth=0.2,
                        figsize=(15, 10))

Now, you do see a clean map.